https://github.com/stellasia/nodes2022
    
https://www.youtube.com/watch?v=DKziks5jQvc&ab_channel=Neo4j

In [1]:


from typing import Optional


class Movie:
    def __init__(self, title: str, released: Optional[int] = None):
        self.title = title
        self.released = released


def build_create_movie_query(movie: Movie):
    query = "CREATE (node:Movie) SET node.title=$title, node.released=$released"
    params = {
        "title": movie.title,
        "released": movie.released
    }
    return query, params

In [11]:
import pytest

def test_movie_object():
    m = Movie(title="The Matrix")
    assert m.title == "The Matrix"
    assert m.released is None



def movie():
    return Movie(title="NODES", released=2022)


def test_movie_create_query(movie):
    q, p = build_create_movie_query(movie)
    assert q == "CREATE (node:Movie) SET node.title=$title, node.released=$released"
    assert p == {"title": movie.title, "released": movie.released}

In [12]:
test_movie_object()

In [13]:
a = movie()

In [15]:
a.released

2022

## Pilot


Node File: 
* Name
* Date Created
* Date Modified
* Format

Node Keyword:

* Name

Relationship:

* PER
* LOC
* ORG
* MIS

Check if a node exists:
* If yes, update.
* Else Create. 

Syntax: Create:

CREATE (a {name: 'Andy'})
RETURN a.name

CREATE (n:Person {name: 'Andy', title: 'Developer'})

Syntax Delete:
MATCH (m:Person {name: 'Mark'})
DETACH DELETE m


In [13]:
class Helper:
    
    def formatProperties(self, properties):
        
        query = "{"
        
        for key,val in properties.items():      
            query += key
            query += ":'"
            query += val + "',"
        query = query[:-1] + "}"
        
        return query
            
Help = Helper() 

In [154]:
class File:
    
    def __init__(self, name: str, date_created, date_modified, file_format):
        
        self.properties = {}
        self.properties['name'] = name
        self.properties['date_created'] = date_created
        self.properties['date_modified'] = date_modified
        self.properties['file_format'] = file_format
        self.type = "Files"
        
    def getProperties(self):
        
        return Help.formatProperties(self.properties)
        
        
class Keyword:
    
    def __init__(self, name, prop):
        
        self.properties = {}
        self.properties['name'] = name
        self.properties['property'] = prop
        self.type = 'Keywords'
        
    def getProperties(self):
        return Help.formatProperties(self.properties)
    
    def getRelationship(self):
        return self.relationship
        

        

In [83]:
class QueryGenerator:
    
    def __init__(self, conn = ""):
        self.conn = conn

    def delete(self, node: str, properties: str):
        
        query = "MATCH (m:" + node + " " + properties + ") DETACH DELETE m"
        return query

    def create(self, node: str, properties: str):
        
        query = "CREATE (n:" + node + " " + properties + ")"
        return query
    
    def update(self, node: str, properties: str):
        
        return self.delete(node, properties) + " " + self.create(node, properties)

    def check(self, node: str, properties: str):
        return 
    
    def connect_nodes(self, node1, node2, relationship):
        
        query = "MATCH (n1:" + node1.type + node1.getProperties() + "),"
        query += "(n2:" + node2.type + node2.getProperties() + ") "
        query += "CREATE (n1) - [:" + relationship + "] -> (n2)"
    
        return query
    
    def relationship_exists(self, node1, node2, relationship):
        
        query = "RETURN EXISTS( (:"+ node1.type + "{ name: '"+ node1.properties['name'] + "'})-[:"+ relationship + "]-"
        query += "(: " + node2.type + "{name: '" + node2.properties['name'] +"'}) )"
        
        return query
        
        

In [136]:
from py2neo import Graph


class Neo4jConnection:
    
    '''
    Get BOLT URL, username, pwd
    Create Session.
    Run queries through session.
    '''
    
    def __init__(self):
        self.uri = "bolt://localhost:7687"
        self.user = "neo4j"
        self.password = "neo4jneo4j"
        self.graph = None
        self.queryGenerator = QueryGenerator()
        
    def connect(self):
        
        self.graph = Graph(self.uri, auth=(self.user, self.password))
        return self.graph
        
    
    def run(self, node1, func=None, node2=None, relationship=None):
        
        
        query = ""
        
        if func == "CREATE":
            
            if self.exists(node1):
                print("Node Exists. Cannot create nodes with redundant names.")
                return -1
            
            query = self.queryGenerator.create(node1.type, node1.getProperties())
            
      
        elif func == "DELETE":
            
            if self.exists(node1) == False:
                print("Node doesn't exist")
                return -1
            
            query = self.queryGenerator.delete(node1.type, node1.getProperties())
            
        elif func == "UPDATE":
            pass
            
        elif func == "LINK":
            
            if node2 == None:
                print("Node2 Argument Missing")
                return -1
            if self.exists(node1) == False:
                print("Create Node 1")
                return -1
            if self.exists(node2) == False:
                print("Create Node 2")
                return -1
            if relationship == None:
                print("Add relationship")
                return -1
             
            if graphConn.graph.run(self.queryGenerator.relationship_exists(node1, node2, relationship)).evaluate():
                print("Relationship exists")
                return -1
                
            query = self.queryGenerator.connect_nodes(node1, node2, relationship)
            
        else:
            
            print("Unknown Function. Choose one of the following: [CREATE, DELETE, UPDATE, LINK]")
            return -1
            
        
        try:
            return self.graph.run(query)
        except:
            print("Query ERROR!")
            return -1
    
    def exists(self, node):
        
        count = self.graph.nodes.match(node.type, name=node.properties['name']).first()
        
        return False if count == None else True
        
         
        

In [ ]:
file_node = File("Ram", "123","456","pdf")
entity_node = Keyword("Chennai","Location")
entity_node2 = Keyword("San Jose","Location")

In [156]:
file_node.getProperties()

"{name:'Ram',date_created:'123',date_modified:'456',file_format:'pdf'}"

In [157]:
entity_node.getProperties()

"{name:'Chennai',property:'Location'}"

In [158]:
graphConn = Neo4jConnection()

In [159]:
graphConn.connect()

Graph('bolt://localhost:7687')

In [160]:
entity_node.getProperties()


"{name:'Chennai',property:'Location'}"

In [161]:
graphConn.run(file_node, "CREATE")

In [163]:
graphConn.run(entity_node, "CREATE")

In [164]:
graphConn.run(entity_node2, "CREATE")

In [166]:
graphConn.run(file_node, "LINK",entity_node, "FROM")

In [167]:
graphConn.run(file_node, "LINK",entity_node2, "LIVES")